In [79]:
import numpy as np
import pandas as pd 
df=pd.read_csv("new_data.csv")

In [80]:
df

,Thread Number,Spatial Distribution,Temporal Distribution,T/R,Processor Utilization,Channel Waiting Time,Input Waiting Time,Network Response Time,Channel Utilization
0,839546,1,974686,308,491814,700,514102,0,352431
1,827412,9,556437,291,37663,864,599227,0,506302
2,802605,27,27618,264,928002,839,372851,0,638516
3,723403,61,848511,235,776888,1256,53108,0,767051
4,72121,121,85884,189,680044,1343,875577,0,807812
...,...,...,...,...,...,...,...,...,...
570,906607,970,432438,485,592935,307,792664,0,941146
571,79234,532,737732,363,94043,1214,57768,0,961042
572,707199,751,67511,224,638088,1653,736882,0,981944
573,61823,905,326843,206,922631,1421,304936,0,9625


In [81]:
df['Network Response Time'].unique()

array([   0,  607,  902, 1042, 1219, 1202, 1688, 1904, 2191, 2752],
      dtype=int64)

In [82]:
df = df.reindex(columns=['Thread Number','Spatial Distribution','Temporal Distribution','T/R','Processor Utilization','Channel Waiting Time','Input Waiting Time','Channel Utilization','Network Response Time'])

In [83]:
tar_map = {0: 0, 607: 1, 902: 2, 1042: 3, 1219: 4, 1202: 5, 1688: 6, 1904: 7, 2191: 8, 2752: 9}
df['Network Response Time'] = df['Network Response Time'].map(tar_map)  

In [84]:
df

,Thread Number,Spatial Distribution,Temporal Distribution,T/R,Processor Utilization,Channel Waiting Time,Input Waiting Time,Channel Utilization,Network Response Time
0,839546,1,974686,308,491814,700,514102,352431,0
1,827412,9,556437,291,37663,864,599227,506302,0
2,802605,27,27618,264,928002,839,372851,638516,0
3,723403,61,848511,235,776888,1256,53108,767051,0
4,72121,121,85884,189,680044,1343,875577,807812,0
...,...,...,...,...,...,...,...,...,...
570,906607,970,432438,485,592935,307,792664,941146,0
571,79234,532,737732,363,94043,1214,57768,961042,0
572,707199,751,67511,224,638088,1653,736882,981944,0
573,61823,905,326843,206,922631,1421,304936,9625,0


In [85]:
cor = df.corrwith(df['Network Response Time']).abs()

sel_features = cor.nlargest(6).index.tolist()[1:]

# Print the names of the selected features
print("Positively correlated Features are: ", sel_features)

Positively correlated Features are:  ['Channel Waiting Time', 'T/R', 'Thread Number', 'Processor Utilization', 'Channel Utilization']


In [86]:
X = df.iloc[:, 1:-1]
#X = df[['Channel Waiting Time', 'T/R', 'Thread Number', 'Processor Utilization', 'Channel Utilization']]
y = df.iloc[:,-1] 

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [89]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [90]:
# Define the pipeline steps
scaler = StandardScaler() # Standardize the features
pca = PCA(n_components=5) # Reduce the dimensionality with PCA
reg = LinearRegression() # Fit a linear regression model

# Create the pipeline object
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("reg", reg)])

# Fit the pipeline on the training data
pipe.fit(X_train, y_train)

# Predict on the test data
y_pred = pipe.predict(X_test)
print("Accuracy score =", round(sm.r2_score(y_test, y_pred), 2)*100,"%")

Accuracy score = -4.0 %


In [92]:
# Define the pipeline steps
scaler = StandardScaler() # Standardize the features
pca = PCA(n_components=5) # Reduce the dimensionality with PCA
svr = SVR(kernel="rbf", C=100) # Fit a SVM model

# Create the pipeline object
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("svr", svr)])

# Fit the pipeline on the training data
pipe.fit(X_train, y_train)

# Predict on the test data
y_pred = pipe.predict(X_test)
print("Accuracy score =", round(sm.r2_score(y_test, y_pred), 2)*100,"%")

Accuracy score = -3.0 %


In [94]:
# Define the pipeline steps
scaler = StandardScaler() # Standardize the features
pca = PCA(n_components=2) # Reduce the dimensionality with PCA
knn = KNeighborsRegressor(n_neighbors=5, metric="euclidean") # Fit a KNN model

# Create the pipeline object
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

# Fit the pipeline on the training data
pipe.fit(X_train, y_train)

# Predict on the test data
y_pred = pipe.predict(X_test)
print("Accuracy score =", round(sm.r2_score(y_test, y_pred), 2)*100,"%")

Accuracy score = -6.0 %
